In [1]:
import pandas as pd
import numpy as np
import urllib.request
import requests


# 1

### with requests

In [ ]:
"""
# Set up API keys
DISTANCE_MATRIX_API_KEY = ''
GEOCODE_API_KEY = ''

# Set up base URLs for APIs
DISTANCE_MATRIX_BASE_URL = 'https://maps.googleapis.com/maps/api/distancematrix/json'
GEOCODE_BASE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

# Set up origin city
origin = 'Tel Aviv'

# Read destinations from file
with open('dests.txt', 'r') as f:
    destinations = f.read().splitlines()

# Set up empty lists to store data
targets = []
distances = []
durations = []
longitudes = []
latitudes = []

# Loop through destinations and retrieve data
for destination in destinations:
    # Get distance and duration data from Distance Matrix API
    distance_matrix_params = {
        'origins': origin,
        'destinations': destination,
        'key': DISTANCE_MATRIX_API_KEY,
        'units': 'metric'
    }
    distance_matrix_response = requests.get(DISTANCE_MATRIX_BASE_URL, params=distance_matrix_params).json()
    distance = distance_matrix_response['rows'][0]['elements'][0]['distance']['value'] / 1000
    duration = distance_matrix_response['rows'][0]['elements'][0]['duration']['value'] / 60

    # Get longitude and latitude data from Geocode API
    geocode_params = {
        'address': destination,
        'key': GEOCODE_API_KEY
    }
    geocode_response = requests.get(GEOCODE_BASE_URL, params=geocode_params).json()
    longitude = geocode_response['results'][0]['geometry']['location']['lng']
    latitude = geocode_response['results'][0]['geometry']['location']['lat']

    # Append data to lists
    targets.append(destination)
    distances.append(distance)
    durations.append(duration)
    longitudes.append(longitude)
    latitudes.append(latitude)

# Create DataFrame from data
df = pd.DataFrame({
    'Target': targets,
    'Distance_km': distances,
    'Duration(M)': durations,
    'Longitude': longitudes,
    'Latitude': latitudes
})
"""


### with urllib

In [14]:

# Replace with your own API key
api_key = 'Replace with your own API key'

# Read destinations from file
with open('dests.txt', 'r') as f:
    destinations = f.read().splitlines()

# Initialize empty lists to store data
targets = []
distances = []
durations = []
longitudes = []
latitudes = []

# Loop over destinations
for destination in destinations:
    # Encode destination for use in URL
    destination_encoded = urllib.parse.quote(destination)

    # Distance Matrix API request
    distance_matrix_url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins=Tel+Aviv&destinations={destination_encoded}&key={api_key}'
    with urllib.request.urlopen(distance_matrix_url) as response:
        data = json.load(response)
        if data['status'] == 'OK':
            row = data['rows'][0]['elements'][0]
            distance_km = row['distance']['value'] / 1000
            duration_min = row['duration']['value'] / 60
        else:
            print(f'The connection to distance_matrix for {destination} failed')

    # Geocoding API request
    geocode_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={destination_encoded}&key={api_key}'
    with urllib.request.urlopen(geocode_url) as response:
        data = json.load(response)
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            longitude = location['lng']
            latitude = location['lat']
        else:
            print(f'The connection to geocode for {destination} failed')
        

    # Append data to lists
    targets.append(destination)
    distances.append(distance_km)
    durations.append(duration_min)
    longitudes.append(longitude)
    latitudes.append(latitude)

# Create DataFrame
df = pd.DataFrame({
    'Target': targets,
    'Distance_km': distances,
    'Duration(M)': durations,
    'Longitude': longitudes,
    'Latitude': latitudes
})


# 2

In [16]:
# Convert minutes to hours and minutes
df['Duration(HH:MM)'] = df['Duration(M)'].apply(lambda x: f"{int(x) // 60:02d}:{int(x) % 60:02d}")

df

,Target,Distance_km,Duration(M),Longitude,Latitude,Duration(HH:MM)
0,Istanbul,1815.227,1262.866667,28.978359,41.008238,21:02
1,Amsterdam,4532.541,2883.516667,4.904139,52.367573,48:03
2,Valletta,3792.883,3050.933333,14.514100,35.899237,50:50
3,Basel,4092.805,2642.450000,7.588576,47.559599,44:02
4,Doha,2164.477,1358.783333,51.531040,25.285447,22:38


# 3

In [17]:
df.nlargest(3, 'Distance_km')

,Target,Distance_km,Duration(M),Longitude,Latitude,Duration(HH:MM)
1,Amsterdam,4532.541,2883.516667,4.904139,52.367573,48:03
3,Basel,4092.805,2642.450000,7.588576,47.559599,44:02
2,Valletta,3792.883,3050.933333,14.514100,35.899237,50:50
